# Simple Model

- We don't have returns after 2022 Jan. 
- We have data until 2022 Sept.

- For model training:
    - Testing period --> 2021 June - 2022 Jan
    - Train period: 2017 June - 2021 June (3 years)
    One stock per line. Aggregate all data into one row

- For Inference:
    - Infer for periods between 2019 Sept. - 2022 Sept.
    - This will give us the returns for 2022 Sept. until 2023 March. 
    - Choose the top 15 stocks



In [1]:
cd ..

c:\Users\teuta\Desktop\vscode\quinten-stock-prediction


In [2]:
import pandas as pd
import numpy as np
import src.data.preprocessing as pr
import datetime as dt

In [3]:
df = pr.get_data()

c:\Users\teuta\Desktop\vscode\quinten-stock-prediction\src\data\preprocessing.py:7: DtypeWarning: Columns (8,17,26,33,34,66,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,252,253,255,257,259,261,263,265,267,269,270,271,272,273,274,275,276,277,278,279,280,284,285,286,290) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, delimiter=";", parse_dates=date_cols).iloc[:, 1:]
c:\Users\teuta\Desktop\vscode\quinten-stock-prediction\src\data\preprocessing.py:50: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Call ffill before callin

In [4]:
df[df.year==2022].groupby("quarter").apply(lambda x: len(x.index.get_level_values(1).unique()))

quarter
1    1407
3    1407
6    1407
9    1407
dtype: int64

In [5]:
df[df.year==2022].close.dropna()

date        symbol  
2022-01-31  0O9.F        11.952000
            0QF.F       148.268000
            217A.F        7.990000
            22UA.F      156.159998
            2CRSI.PA      4.443000
                           ...    
            XIL.PA       38.820000
            XIOR.BR      48.390000
            YATRA.AS      3.300000
            YK6B.F       15.350000
            ZEL.NZ        3.598000
Name: close, Length: 1387, dtype: float64

# Model: XGBoost 

__Only keep stocks that are trading in Euros
currency_by_stock = df["reportedCurrency"].groupby("symbol").apply(lambda x:x.dropna()[0] if len(x.dropna()) > 0 else np.nan)
euro_stocks = currency_by_stock[currency_by_stock=="EUR"].index
idx = pd.IndexSlice[:, euro_stocks]
df = df.loc[idx, :]

In [6]:
df.index.get_level_values(0)

DatetimeIndex(['1995-01-31', '1995-01-31', '1995-01-31', '1995-01-31',
               '1995-01-31', '1995-01-31', '1995-01-31', '1995-01-31',
               '1995-01-31', '1995-01-31',
               ...
               '2022-09-30', '2022-09-30', '2022-09-30', '2022-09-30',
               '2022-09-30', '2022-09-30', '2022-09-30', '2022-09-30',
               '2022-09-30', '2022-09-30'],
              dtype='datetime64[ns]', name='date', length=157584, freq=None)

In [19]:
# Set train and validation dates
train_dates = {"start": dt.datetime(2019, 6, 30), "end": dt.datetime(2021, 6, 30)}
val_dates = {"start": dt.datetime(2022, 1, 1), "end": dt.datetime(2023, 1, 31)}

X_train = df.loc[train_dates["start"]: train_dates["end"]]
X_train = X_train.unstack(level=0).drop("year", axis=1)
X_train

quarter                                                         \
date     2019-06-30 2019-09-30 2020-01-31 2020-03-31 2020-06-30 2020-09-30   
symbol                                                                       
0O9.F             6          9          1          3          6          9   
0QF.F             6          9          1          3          6          9   
217A.F            6          9          1          3          6          9   
22UA.F            6          9          1          3          6          9   
2CRSI.PA          6          9          1          3          6          9   
...             ...        ...        ...        ...        ...        ...   
XIOR.BR           6          9          1          3          6          9   
YATRA.AS          6          9          1          3          6          9   
YK6B.F            6          9          1          3          6          9   
ZEL.NZ            6          9          1          3          6          9   
ZEN.BR            6          9          1          3          6          9   

                                                week  ...  \
date     2021-01-31 2021-03-31 2021-06-30 2019-06-30  ...   
symbol                                                ...   
0O9.F             1          3          6         26  ...   
0QF.F             1          3          6         26  ...   
217A.F            1          3          6         26  ...   
22UA.F            1          3          6         26  ...   
2CRSI.PA          1          3          6         26  ...   
...             ...        ...        ...        ...  ...   
XIOR.BR           1          3          6         26  ...   
YATRA.AS          1          3          6         26  ...   
YK6B.F            1          3          6         26  ...   
ZEL.NZ            1          3          6         26  ...   
ZEN.BR            1          3          6         26  ...   

         market_value_per_share   pe_ratio                                    \
date                 2021-06-30 2019-06-30 2019-09-30 2020-01-31  2020-03-31   
symbol                                                                         
0O9.F                       NaN        NaN        NaN        NaN         NaN   
0QF.F                       NaN        NaN        NaN        NaN         NaN   
217A.F                19.200001  -7.471373 -11.326101        NaN  -10.052288   
22UA.F               189.250000        NaN -99.093458        NaN -214.101854   
2CRSI.PA                    NaN        NaN        NaN        NaN         NaN   
...                         ...        ...        ...        ...         ...   
XIOR.BR                     NaN        NaN        NaN        NaN         NaN   
YATRA.AS                    NaN        NaN        NaN        NaN -127.790430   
YK6B.F                      NaN        NaN        NaN        NaN         NaN   
ZEL.NZ                      NaN        NaN        NaN        NaN         NaN   
ZEN.BR                      NaN        NaN        NaN        NaN         NaN   

                                                                   
date      2020-06-30 2020-09-30 2021-01-31  2021-03-31 2021-06-30  
symbol                                                             
0O9.F            NaN        NaN        NaN         NaN        NaN  
0QF.F            NaN        NaN        NaN         NaN        NaN  
217A.F     -1.386686  22.112379        NaN  -46.419172   6.104760  
22UA.F   -159.552784 -66.863636        NaN   19.719828  16.571804  
2CRSI.PA         NaN        NaN        NaN         NaN        NaN  
...              ...        ...        ...         ...        ...  
XIOR.BR          NaN        NaN        NaN         NaN        NaN  
YATRA.AS         NaN        NaN        NaN -103.920325        NaN  
YK6B.F           NaN        NaN        NaN         NaN        NaN  
ZEL.NZ           NaN        NaN        NaN         NaN        NaN  
ZEN.BR           NaN        NaN        NaN         NaN     

In [21]:
# y_train = 
test_per_rets = df.loc[val_dates["start"]: val_dates["end"], "return"]
test_per_rets.value_counts()

return
 0.000000    4242
-0.041667       2
 0.090909       2
-0.214286       2
 0.230950       1
             ... 
 0.031020       1
 0.376026       1
-0.018137       1
 0.526165       1
 0.055132       1
Name: count, Length: 1375, dtype: int64

In [22]:
# drop stocks with missing returns for this period
to_drop = test_per_rets[test_per_rets.isna()].index.get_level_values("symbol").unique()

# drop stocks that traded under 1$ in the period
under_1 = X_train['close'].groupby(level="symbol").apply(lambda x: (x<1).any(axis=1)).droplevel(1)
to_drop = to_drop.union(under_1[under_1].index).unique()

X_train = X_train.drop(to_drop)
y_train = test_per_rets.drop(to_drop, level=1)

In [10]:
print(np.setdiff1d(X_train.index, y_train.index.get_level_values(1)))
print(np.setdiff1d(y_train.index.get_level_values(1), X_train.index))

[]
[]


In [11]:
y_train = y_train.groupby(level=1, group_keys=False).apply(lambda x: x.cumprod()[-1])

assert all(X_train.index == y_train.index)

C:\Users\teuta\AppData\Local\Temp\ipykernel_16956\2731682971.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train = y_train.groupby(level=1, group_keys=False).apply(lambda x: x.cumprod()[-1])


In [12]:
# Last checks and drops on columns
X_train = X_train.select_dtypes(exclude=["object", "datetime"])

In [13]:
X_train.dtypes.value_counts()


float64    2547
int64        18
Name: count, dtype: int64

In [14]:
from xgboost import XGBRegressor

In [15]:
model = XGBRegressor()
model.fit(X_train.to_numpy(), y_train)

c:\Users\teuta\Desktop\vscode\.venv\Lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [16]:
#model.save_model()

In [17]:
model.score(X_train, y_train)

c:\Users\teuta\Desktop\vscode\.venv\Lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\teuta\Desktop\vscode\.venv\Lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)


c:\Users\teuta\Desktop\vscode\.venv\Lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\teuta\Desktop\vscode\.venv\Lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dtype(dtype)


0.9991523339370572

# Predicting Returns

In [18]:
infer_dates = {"start": dt.datetime(2018, 6, 30), "end": dt.datetime(2020, 6, 30)}
predicted_dates = {"start": dt.datetime(2020, 6, 30), "end": dt.datetime(2021, 1, 31)}

X_test = df.loc[infer_dates["start"]: infer_dates["end"]]
X_test = X_test.unstack(level=0).drop("year", axis=1)
X_test

quarter                                                         \
date     2018-06-30 2018-09-30 2019-01-31 2019-03-31 2019-06-30 2019-09-30   
symbol                                                                       
0O9.F             6          9          1          3          6          9   
0QF.F             6          9          1          3          6          9   
217A.F            6          9          1          3          6          9   
22UA.F            6          9          1          3          6          9   
2CRSI.PA          6          9          1          3          6          9   
...             ...        ...        ...        ...        ...        ...   
XIOR.BR           6          9          1          3          6          9   
YATRA.AS          6          9          1          3          6          9   
YK6B.F            6          9          1          3          6          9   
ZEL.NZ            6          9          1          3          6          9   
ZEN.BR            6          9          1          3          6          9   

                                                week  ...  \
date     2020-01-31 2020-03-31 2020-06-30 2018-06-30  ...   
symbol                                                ...   
0O9.F             1          3          6         26  ...   
0QF.F             1          3          6         26  ...   
217A.F            1          3          6         26  ...   
22UA.F            1          3          6         26  ...   
2CRSI.PA          1          3          6         26  ...   
...             ...        ...        ...        ...  ...   
XIOR.BR           1          3          6         26  ...   
YATRA.AS          1          3          6         26  ...   
YK6B.F            1          3          6         26  ...   
ZEL.NZ            1          3          6         26  ...   
ZEN.BR            1          3          6         26  ...   

         market_value_per_share   pe_ratio                                    \
date                 2020-06-30 2018-06-30 2018-09-30 2019-01-31  2019-03-31   
symbol                                                                         
0O9.F                       NaN        NaN        NaN        NaN         NaN   
0QF.F                       NaN        NaN        NaN        NaN         NaN   
217A.F                20.799999  -9.446522 -33.968887        NaN  137.356871   
22UA.F                60.630058        NaN        NaN        NaN         NaN   
2CRSI.PA                    NaN        NaN        NaN        NaN         NaN   
...                         ...        ...        ...        ...         ...   
XIOR.BR                     NaN        NaN        NaN        NaN         NaN   
YATRA.AS                    NaN        NaN        NaN        NaN    8.230335   
YK6B.F                      NaN        NaN        NaN        NaN         NaN   
ZEL.NZ                      NaN        NaN        NaN        NaN         NaN   
ZEN.BR                      NaN        NaN        NaN        NaN         NaN   

                                                                   
date     2019-06-30 2019-09-30 2020-01-31  2020-03-31  2020-06-30  
symbol                                                             
0O9.F           NaN        NaN        NaN         NaN         NaN  
0QF.F           NaN        NaN        NaN         NaN         NaN  
217A.F    -7.471373 -11.326101        NaN  -10.052288   -1.386686  
22UA.F          NaN -99.093458        NaN -214.101854 -159.552784  
2CRSI.PA        NaN        NaN        NaN         NaN         NaN  
...             ...        ...        ...         ...         ...  
XIOR.BR         NaN        NaN        NaN         NaN         NaN  
YATRA.AS        NaN        NaN        NaN -127.790430         NaN  
YK6B.F          NaN        NaN        NaN         NaN         NaN  
ZEL.NZ          NaN        NaN        NaN         NaN         NaN  
ZEN.BR          NaN        NaN        NaN         NaN      

In [19]:
X_test = X_test.select_dtypes(exclude=["object", "datetime"])
assert len(X_train.columns) == len(X_test.columns)

In [20]:
y_pred = pd.Series(model.predict(X_test), X_test.index)
top20_stocks = y_pred.sort_values()[-20:].index

c:\Users\teuta\Desktop\vscode\.venv\Lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\teuta\Desktop\vscode\.venv\Lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\teuta\Desktop\vscode\.venv\Lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\teuta\Desktop\vscode\.venv\Lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool 

In [21]:
y_test = df.loc[predicted_dates["start"]: predicted_dates["end"], "return"]
y_test = y_test.groupby(level=1, group_keys=False).apply(lambda x: x.cumprod()[-1])
y_test = y_test[y_test.index.isin(y_pred.index)]

C:\Users\teuta\AppData\Local\Temp\ipykernel_20540\1055454902.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_test = y_test.groupby(level=1, group_keys=False).apply(lambda x: x.cumprod()[-1])


In [22]:
def rmse(y_true, y_pred):
    """
    Calculate the Root Mean Squared Error (RMSE).

    Parameters:
    y_true (array-like): Actual (true) values.
    y_pred (array-like): Predicted values.

    Returns:
    float: The RMSE.
    """
    squared_errors = (y_true - y_pred) ** 2
    mean_squared_error = squared_errors.mean()
    rmse = np.sqrt(mean_squared_error)
    return rmse

In [23]:
rmse(y_test, y_pred)

0.19169965319687007

In [24]:
preds = pd.DataFrame({"y_test":y_test, "y_pred": y_pred})#.loc[top20_stocks]

In [25]:
preds["diff"] = preds["y_test"] - preds["y_pred"]
# preds.iloc[np.where((preds["diff"] < 0.01) & (preds["diff"] > -0.01))]
preds.loc[top20_stocks].to_csv("top20_stocks.csv")

In [26]:
import os
os.getcwd()

'c:\\Users\\teuta\\Desktop\\vscode\\quinten-stock-prediction'

In [27]:
from src.models import train
from src.models import inference

In [28]:
start_train = dt.datetime.strptime("2016/01/31", "%Y/%m/%d")


In [29]:
start_train = dt.datetime.strptime("2016/01/31", "%Y/%m/%d")
end_train = dt.datetime.strptime("2018/01/31", "%Y/%m/%d")
start_val = dt.datetime.strptime("2018/01/31", "%Y/%m/%d")
end_val = dt.datetime.strptime("2018/06/30", "%Y/%m/%d")


train.train(start_train, end_train, start_val, end_val)

c:\Users\teuta\Desktop\vscode\quinten-stock-prediction\src\data\preprocessing.py:7: DtypeWarning: Columns (8,17,26,33,34,66,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,252,253,255,257,259,261,263,265,267,269,270,271,272,273,274,275,276,277,278,279,280,284,285,286,290) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, delimiter=";", parse_dates=date_cols).iloc[:, 1:]
c:\Users\teuta\Desktop\vscode\quinten-stock-prediction\src\data\preprocessing.py:50: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Call ffill before callin

# Backtest

In [6]:
cd ..

c:\Users\teuta\Desktop\vscode\quinten-stock-prediction


In [7]:
from dateutil.relativedelta import relativedelta
from src.models import train
from src.models import inference
import src.data.preprocessing as pr
import datetime as dt
import pandas as pd

df = pr.get_data()

c:\Users\teuta\Desktop\vscode\quinten-stock-prediction\src\data\preprocessing.py:7: DtypeWarning: Columns (8,17,26,33,34,66,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,252,253,255,257,259,261,263,265,267,269,270,271,272,273,274,275,276,277,278,279,280,284,285,286,290) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, delimiter=";", parse_dates=date_cols).iloc[:, 1:]
c:\Users\teuta\Desktop\vscode\quinten-stock-prediction\src\data\preprocessing.py:50: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Call ffill before callin

In [8]:
# get dates starting from 2015 Jan
train_duration_months = 24
validation_duration_months = 6
gap = 0

train_start_dates = [dt.datetime(2015, 1, 31) + relativedelta(months=validation_duration_months*i) for i in range(5)]
train_start_dates = [i - relativedelta(months=1) if i.month==7 else i  for i in train_start_dates]
train_end_dates = [i + relativedelta(months=24) for i in train_start_dates]
val_start_dates = [i for i in train_end_dates]
#val_start_dates = [i + relativedelta(months=1) if i.month==5 else i for i in val_start_dates]
val_end_dates = [i + relativedelta(months=validation_duration_months) for i in val_start_dates]
val_end_dates = [i + relativedelta(months=1) if i.month==12 else i - relativedelta(months=1) for i in val_end_dates]

print(train_start_dates, "\n", train_end_dates, "\n", val_start_dates,"\n" , val_end_dates)


inference_start_dates = [i + relativedelta(months=6) for i in train_end_dates]
inference_start_dates = [i - relativedelta(months=1) if i.month == 7 else i+relativedelta(months=1) for i in inference_start_dates]
inference_end_dates = [i + relativedelta(months=24) for i in inference_start_dates]
inference_end_dates = [i + relativedelta(months=1) if i.month == 12 else i for i in inference_end_dates]

predicted_start_dates = [i + relativedelta(months=6) for i in inference_end_dates]
predicted_start_dates = [i - relativedelta(months=1) if i.month == 7 else i+relativedelta(months=1) for i in predicted_start_dates]
predicted_end_dates = [i + relativedelta(months=6) for i in predicted_start_dates]
predicted_end_dates = [i - relativedelta(months=1) if i.month == 7 else i+relativedelta(months=1) for i in predicted_end_dates]



[datetime.datetime(2015, 1, 31, 0, 0), datetime.datetime(2015, 6, 30, 0, 0), datetime.datetime(2016, 1, 31, 0, 0), datetime.datetime(2016, 6, 30, 0, 0), datetime.datetime(2017, 1, 31, 0, 0)] 
 [datetime.datetime(2017, 1, 31, 0, 0), datetime.datetime(2017, 6, 30, 0, 0), datetime.datetime(2018, 1, 31, 0, 0), datetime.datetime(2018, 6, 30, 0, 0), datetime.datetime(2019, 1, 31, 0, 0)] 
 [datetime.datetime(2017, 1, 31, 0, 0), datetime.datetime(2017, 6, 30, 0, 0), datetime.datetime(2018, 1, 31, 0, 0), datetime.datetime(2018, 6, 30, 0, 0), datetime.datetime(2019, 1, 31, 0, 0)] 
 [datetime.datetime(2017, 6, 30, 0, 0), datetime.datetime(2018, 1, 30, 0, 0), datetime.datetime(2018, 6, 30, 0, 0), datetime.datetime(2019, 1, 30, 0, 0), datetime.datetime(2019, 6, 30, 0, 0)]


In [9]:
print(inference_start_dates)
print(inference_end_dates)
print(predicted_start_dates)
print(predicted_end_dates)

[datetime.datetime(2017, 6, 30, 0, 0), datetime.datetime(2018, 1, 30, 0, 0), datetime.datetime(2018, 6, 30, 0, 0), datetime.datetime(2019, 1, 30, 0, 0), datetime.datetime(2019, 6, 30, 0, 0)]
[datetime.datetime(2019, 6, 30, 0, 0), datetime.datetime(2020, 1, 30, 0, 0), datetime.datetime(2020, 6, 30, 0, 0), datetime.datetime(2021, 1, 30, 0, 0), datetime.datetime(2021, 6, 30, 0, 0)]
[datetime.datetime(2020, 1, 30, 0, 0), datetime.datetime(2020, 6, 30, 0, 0), datetime.datetime(2021, 1, 30, 0, 0), datetime.datetime(2021, 6, 30, 0, 0), datetime.datetime(2022, 1, 30, 0, 0)]
[datetime.datetime(2020, 6, 30, 0, 0), datetime.datetime(2021, 1, 30, 0, 0), datetime.datetime(2021, 6, 30, 0, 0), datetime.datetime(2022, 1, 30, 0, 0), datetime.datetime(2022, 6, 30, 0, 0)]


In [18]:
results = pd.DataFrame(columns=["y_test", "y_pred"])
for i in range(len(train_start_dates)):
    model_name = f"{str(train_start_dates[i])[:10]}: {str(train_end_dates[i])[:10]}"
    model = train.train(train_start_dates[i], train_end_dates[i], val_start_dates[i], val_end_dates[i],
        model_name=model_name)
    
    res = inference.inference(inference_start_dates[i], inference_end_dates[i], predicted_start_dates[i], predicted_end_dates[i],
                        model, df=df)
    results = pd.concat([results, res.reset_index(drop=True)])


c:\Users\teuta\Desktop\vscode\quinten-stock-prediction\src\data\preprocessing.py:7: DtypeWarning: Columns (8,17,26,33,34,66,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,252,253,255,257,259,261,263,265,267,269,270,271,272,273,274,275,276,277,278,279,280,284,285,286,290) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, delimiter=";", parse_dates=date_cols).iloc[:, 1:]
c:\Users\teuta\Desktop\vscode\quinten-stock-prediction\src\data\preprocessing.py:50: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Call ffill before callin

ValueError: Feature shape mismatch, expected: 2565, got 2280

In [19]:
results

,y_test,y_pred
0,NaN,0.0
1,-0.004512,0.0
2,0.156022,0.0
3,-0.003902,0.0
4,-0.001176,0.0
5,-0.000453,0.0
6,0.000968,0.0
7,0.003350,0.0
8,-0.002042,0.0
9,-0.197800,0.0
